In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
from getpass import getpass

if not os.getenv("OPENAI_API_KEY"):
    print("🔑 OPENAI_API_KEY 입력 필요")
    key = getpass("API Key: ").strip()
    if key:
        os.environ["OPENAI_API_KEY"] = key
        print("✅ API Key 설정 완료")
else:
    print("✅ API Key 확인됨")


In [ ]:
import operator
from typing import Literal, Annotated, List
from typing_extensions import TypedDict, NotRequired

# LangGraph 핵심 컴포넌트
from langgraph.graph.message import add_messages
# LangChain 메시지 타입
from langchain_core.messages import AnyMessage, AIMessage


class TraderState(TypedDict):
    """
    트레이딩 봇의 전체 상태를 관리하는 TypedDict 클래스

    LangGraph에서 노드 간에 전달되는 상태 정보를 정의합니다.
    각 필드는 트레이딩 결정과 실행 과정에서 필요한 데이터를 담습니다.
    """
    # --- 도메인 상태 ---
    ticker: str  # 거래할 주식 심볼 (예: "AAPL", "GOOGL")
    action: NotRequired[Literal["BUY", "SELL", "HOLD"]]  # 결정된 거래 행동 (선택사항)
    trend: NotRequired[Literal["up", "down", "flat"]]  # 시장 트렌드 분석 결과 (선택사항)
    approved: NotRequired[bool]  # 거래 승인 여부 (선택사항)

    # --- 로깅 및 메시징 시스템 ---
    # Annotated[데이터타입, 리듀서함수]: 상태 업데이트 시 어떻게 병합할지 지정
    logs: Annotated[List[str], operator.add]  # 로그 메시지들을 리스트에 누적 추가
    messages: Annotated[List[AnyMessage], add_messages]  # LangChain 메시지들을 대화 히스토리에 누적 추가


def log(msg: str) -> dict:
    """
    로그 메시지를 상태에 추가하는 헬퍼 함수

    Args:
        msg: 기록할 로그 메시지

    Returns:
        TraderState 형태의 부분 업데이트 딕셔너리
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_log = f"[{timestamp}] {msg}"
    return {"logs": [formatted_log]}


def say(msg: str) -> dict:
    """
    AI 메시지를 대화 히스토리에 추가하는 헬퍼 함수

    Args:
        msg: AI가 말할 메시지 내용

    Returns:
        TraderState 형태의 부분 업데이트 딕셔너리
    """
    return {"messages": [AIMessage(content=msg)]}


# --- 거래 실행 노드들 ---
def execute_buy(state: TraderState) -> dict:
    """
    매수 주문을 실행하는 노드

    Args:
        state: 현재 트레이딩 상태 (ticker 정보 필요)

    Returns:
        로그와 메시지가 업데이트된 상태 딕셔너리
    """
    msg = f"BUY executed for {state['ticker']}"
    return {**log(msg), **say(msg)}


def execute_sell(state: TraderState) -> dict:
    """
    매도 주문을 실행하는 노드

    Args:
        state: 현재 트레이딩 상태 (ticker 정보 필요)

    Returns:
        로그와 메시지가 업데이트된 상태 딕셔너리
    """
    msg = f"SELL executed for {state['ticker']}"
    return {**log(msg), **say(msg)}


def execute_hold(state: TraderState) -> dict:
    """
    보유 결정을 실행하는 노드 (거래하지 않음)

    Args:
        state: 현재 트레이딩 상태 (ticker 정보 필요)

    Returns:
        로그와 메시지가 업데이트된 상태 딕셔너리
    """
    msg = f"HOLD for {state['ticker']}"
    return {**log(msg), **say(msg)}


# -- Logging Helper -- #
import json
import dataclasses
from datetime import date, datetime
from uuid import UUID

from langchain_core.load import dumpd  # 범용 LangChain 직렬화
from langchain_core.messages import BaseMessage
from langchain_core.messages.base import message_to_dict, messages_to_dict


def _safe_primitive(x):
    if isinstance(x, (datetime, date)):
        return x.isoformat()
    if isinstance(x, UUID):
        return str(x)
    if isinstance(x, bytes):
        return x.decode("utf-8", errors="replace")
    if isinstance(x, set):
        return list(x)
    if dataclasses.is_dataclass(x):
        return dataclasses.asdict(x)
    if hasattr(x, "model_dump"):  # Pydantic v2
        return x.model_dump()
    return str(x)


def to_jsonable(obj):
    # 1) LangChain 메시지들
    if isinstance(obj, BaseMessage):
        return message_to_dict(obj)
    if isinstance(obj, (list, tuple)):
        # 리스트/튜플이 메시지들인 경우
        if obj and all(isinstance(m, BaseMessage) for m in obj):
            return messages_to_dict(list(obj))
        # 일반 튜플은 JSON이 안되니 리스트로 풀어줌
        return [to_jsonable(x) for x in obj]

    # 2) LangChain 객체 전반: dumpd가 알아서 처리
    try:
        dumped = dumpd(obj)
        # dumpd가 그대로 원시(dict/list/str...)를 반환하면 ok
        if isinstance(dumped, (dict, list, str, int, float, bool)) or dumped is None:
            return dumped
    except Exception:
        pass

    # 3) 남은 케이스(날짜/UUID/bytes/set/dataclass/pydantic 등)
    return _safe_primitive(obj)


In [ ]:
import random
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


def random_decider(state: TraderState) -> dict:
    """
    랜덤하게 거래 행동을 결정하는 노드

    실제 트레이딩 봇에서는 시장 분석, 기술적 지표 등을 사용하겠지만,
    여기서는 데모 목적으로 무작위로 BUY/SELL/HOLD 중 하나를 선택합니다.

    Args:
        state: 현재 트레이딩 상태

    Returns:
        결정된 action과 로그가 포함된 상태 업데이트 딕셔너리
    """
    action = random.choice(["BUY", "SELL", "HOLD"])
    return {"action": action, **log(f"[random_decider] -> {action}")}


def route_by_action(state: TraderState) -> Literal["buy", "sell", "hold"]:
    """
    결정된 action에 따라 다음 노드로 라우팅하는 조건부 엣지 함수

    LangGraph의 conditional_edges에서 사용되어 상태의 action 값에 따라
    적절한 실행 노드로 흐름을 분기시킵니다.

    Args:
        state: action 필드를 포함한 현재 상태

    Returns:
        다음에 실행할 노드 이름 ("buy", "sell", "hold" 중 하나)
    """
    return state["action"].lower()


# --- 그래프 구성 ---
# TraderState를 상태로 사용하는 StateGraph 인스턴스 생성
graph_builder = StateGraph(TraderState)

# 노드들을 그래프에 추가
graph_builder.add_node("random_decider", random_decider)  # 의사결정 노드
graph_builder.add_node("buy", execute_buy)  # 매수 실행 노드
graph_builder.add_node("sell", execute_sell)  # 매도 실행 노드
graph_builder.add_node("hold", execute_hold)  # 보유 실행 노드

# 엣지 연결: 그래프의 실행 흐름 정의
graph_builder.add_edge(START, "random_decider")  # 시작점에서 의사결정 노드로

# 조건부 엣지: random_decider의 결과에 따라 분기
graph_builder.add_conditional_edges(
    "random_decider",  # 소스 노드
    route_by_action,  # 라우팅 함수
    {  # 라우팅 맵: 함수 반환값 -> 대상 노드
        "buy": "buy",
        "sell": "sell",
        "hold": "hold"
    }
)

# 모든 실행 노드에서 END로 연결 (그래프 종료)
for node_name in ["buy", "sell", "hold"]:
    graph_builder.add_edge(node_name, END)

# 그래프 컴파일: 실행 가능한 형태로 변환
trading_graph_v1 = graph_builder.compile()

# 그래프 시각화: Mermaid 다이어그램으로 구조 표시
display(Image(trading_graph_v1.get_graph().draw_mermaid_png()))

In [ ]:
initial_state = {"ticker": "AAPL", "logs": [], "messages": []}

print("\n================ stream_mode='updates' ================")
for chunk in trading_graph_v1.stream(initial_state, stream_mode="updates"):
    print(json.dumps(chunk, ensure_ascii=False, indent=2, default=to_jsonable))
    print("__________________________________________________________")

print("\n================ stream_mode='values' ================")
for chunk in trading_graph_v1.stream(initial_state, stream_mode="values"):
    print(json.dumps(chunk, ensure_ascii=False, indent=2, default=to_jsonable))
    print("__________________________________________________________")

print("\n================ stream_mode='messages' ================")
for chunk in trading_graph_v1.stream(initial_state, stream_mode="messages"):
    print(json.dumps(chunk, ensure_ascii=False, indent=2, default=to_jsonable))
    print("__________________________________________________________")

# messages에 따라 나온 system 정보는 무엇일까? 예시처럼 특정 토큰만 추출하거나 식별하기 좋게 해주는 구조
# 
# for msg, meta in graph.stream(inputs, stream_mode="messages"):
#     if meta.get("langgraph_node") == "buy":
#         print(msg.content, end="", flush=True)
# 


In [ ]:
from random import uniform, choice


def fetch_market_data(state: TraderState) -> dict:
    """
    시장 데이터를 가져와서 현재 가격과 트렌드를 분석하는 노드

    실제 트레이딩 봇에서는 실시간 API나 데이터 피드를 사용하지만,
    데모 목적으로 랜덤한 가격과 트렌드를 생성합니다.

    Args:
        state: 현재 트레이딩 상태 (ticker 정보 사용)

    Returns:
        분석된 트렌드 정보와 로그가 포함된 상태 업데이트 딕셔너리
    """
    # 50~150 범위의 랜덤 주가 생성 (소수점 2자리까지)
    current_price = round(uniform(50, 150), 2)

    # 시장 트렌드 분석 결과 (상승/하락/보합)
    market_trend = choice(["up", "down", "flat"])

    return {
        "trend": market_trend,
        **log(f"[fetch_market] {state.get('ticker', 'UNKNOWN')} price=${current_price}, trend={market_trend}")
    }


def rule_based_decision(state: TraderState) -> dict:
    """
    규칙 기반으로 거래 결정을 내리는 노드

    간단한 트렌드 추종 전략을 사용합니다:
    - 상승 트렌드 → 매수 (BUY)
    - 하락 트렌드 → 매도 (SELL)
    - 보합 트렌드 → 보유 (HOLD)

    Args:
        state: 트렌드 정보가 포함된 현재 상태

    Returns:
        결정된 거래 행동과 로그가 포함된 상태 업데이트 딕셔너리
    """
    current_trend = state.get("trend", "flat")

    # 트렌드 추종 규칙 적용
    if current_trend == "up":
        trading_action = "BUY"
    elif current_trend == "down":
        trading_action = "SELL"
    else:  # flat or unknown
        trading_action = "HOLD"

    return {
        "action": trading_action,
        **log(f"[rule_router] trend={current_trend} -> {trading_action}")
    }


# --- 그래프 구성 ---
# V2: 시장 데이터 분석 → 규칙 기반 의사결정 → 실행 파이프라인
graph_builder_v2 = StateGraph(TraderState)

# 모든 노드를 효율적으로 추가
node_definitions = [
    ("fetch_market", fetch_market_data),  # 시장 데이터 수집
    ("rule_router", rule_based_decision),  # 규칙 기반 의사결정
    ("buy", execute_buy),  # 매수 실행
    ("sell", execute_sell),  # 매도 실행
    ("hold", execute_hold),  # 보유 실행
]

for node_name, node_function in node_definitions:
    graph_builder_v2.add_node(node_name, node_function)

# 순차적 실행 흐름 정의
graph_builder_v2.add_edge(START, "fetch_market")  # 시작 → 시장 데이터 수집
graph_builder_v2.add_edge("fetch_market", "rule_router")  # 데이터 수집 → 의사결정

# 의사결정 결과에 따른 조건부 분기
graph_builder_v2.add_conditional_edges(
    "rule_router",  # 소스: 규칙 기반 의사결정 노드
    route_by_action,  # 라우팅 함수: action 값에 따라 분기
    {  # 분기 맵
        "buy": "buy",
        "sell": "sell",
        "hold": "hold"
    }
)

# 모든 실행 노드에서 그래프 종료
for execution_node in ["buy", "sell", "hold"]:
    graph_builder_v2.add_edge(execution_node, END)

# 실행 가능한 그래프로 컴파일
trading_graph_v2 = graph_builder_v2.compile()
display(Image(trading_graph_v2.get_graph().draw_mermaid_png()))

# --- 데모 실행 ---
print("🤖 Trading Bot Graph V2 - Rule-Based Strategy")
print("=" * 50)

# GOOG 주식에 대한 규칙 기반 거래 시뮬레이션
demo_result = trading_graph_v2.invoke({
    "ticker": "GOOG",
    "logs": [],
    "messages": []
})

print("실행 결과:")
print(json.dumps(demo_result, ensure_ascii=False, indent=2, default=to_jsonable))

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.globals import set_debug
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

set_debug(False)


class TradingDecision(BaseModel):
    """
    LLM의 거래 결정을 구조화하기 위한 Pydantic 모델

    JSON 출력 파싱과 타입 안전성을 보장합니다.
    """
    action: Literal["BUY", "SELL", "HOLD"]
    reason: str = Field(..., description="결정에 대한 간단한 근거")


def create_llm_trading_chain(structured: bool = False):
    """
    LLM 기반 거래 결정 체인을 생성하는 팩토리 함수.

    Args:
        structured (bool): True면 ChatOpenAI.with_structured_output() 사용,
                          False면 JsonOutputParser + Prompt 방식 사용.

    Returns:
        Runnable 체인: Prompt → LLM → (Parser or StructuredOutput)
    """
    if not structured:
        # -------------------------
        # 1. 기존 JsonOutputParser 방식
        # -------------------------
        output_parser = JsonOutputParser(pydantic_object=TradingDecision)
        format_instructions = output_parser.get_format_instructions()

        decision_prompt = PromptTemplate(
            template=(
                "You are a professional trading decision assistant.\n\n"
                "CONTEXT:\n"
                "- Ticker Symbol: {ticker}\n"
                "- Current Market Trend: {trend}\n\n"
                "TASK:\n"
                "Analyze the given information and make a trading decision.\n"
                "Choose EXACTLY one action from ['BUY', 'SELL', 'HOLD'] and provide "
                "a concise rationale for your decision.\n\n"
                "OUTPUT FORMAT:\n"
                "{format_instructions}"
            ),
            input_variables=["ticker", "trend"],
            partial_variables={"format_instructions": format_instructions},
        )

        trading_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
        return decision_prompt | trading_llm | output_parser


    else:
        # -------------------------
        # 2. structured_output 방식
        # -------------------------
        decision_prompt = PromptTemplate(
            template=(
                "You are a professional trading decision assistant.\n\n"
                "Ticker: {ticker}\n"
                "Current Market Trend: {trend}\n\n"
                "Pick EXACTLY one action from ['BUY','SELL','HOLD'] "
                "and provide a concise rationale.\n"
                "Return ONLY JSON with fields: action, reason."
            ),
            input_variables=["ticker", "trend"],
        )

        trading_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0).with_structured_output(TradingDecision)

        return decision_prompt | trading_llm


def llm_powered_decision(state: TraderState) -> dict:
    """
    LLM을 사용하여 거래 결정을 내리는 노드

    시장 데이터를 바탕으로 GPT-4에게 거래 결정을 요청하고,
    구조화된 응답을 받아 상태를 업데이트합니다.

    Args:
        state: 현재 트레이딩 상태 (ticker, trend 정보 필요)

    Returns:
        LLM 결정과 근거가 포함된 상태 업데이트 딕셔너리
    """
    llm_chain = create_llm_trading_chain(structured=True)

    # 현재 트렌드 정보 (없으면 기본값 'flat' 사용)
    current_trend = state.get("trend", "flat")

    try:
        # LLM에게 거래 결정 요청
        llm_result = llm_chain.invoke({
            "ticker": state["ticker"],
            "trend": current_trend
        })

        decision_result = TradingDecision.model_validate(llm_result)

        return {
            "action": decision_result.action,
            **log(f"[llm_decider] {decision_result.reason}")
        }

    except Exception as e:
        # LLM 호출 실패 시 안전한 기본값 반환
        fallback_action = "HOLD"
        error_msg = f"LLM 호출 실패: {str(e)}, 기본값 {fallback_action} 사용"
        return {
            "action": fallback_action,
            **log(f"[llm_decider] {error_msg}")
        }


# --- 그래프 구성 ---
# V3: 시장 데이터 → LLM 의사결정 → 실행 파이프라인
graph_builder_v3 = StateGraph(TraderState)

# 노드 정의: V2의 규칙 기반 의사결정을 LLM으로 교체
node_definitions_v3 = [
    ("fetch_market", fetch_market_data),  # 시장 데이터 수집 (V2와 동일)
    ("llm_decider", llm_powered_decision),  # LLM 기반 의사결정 (NEW!)
    ("buy", execute_buy),  # 매수 실행
    ("sell", execute_sell),  # 매도 실행
    ("hold", execute_hold),  # 보유 실행
]

# 모든 노드 추가
for node_name, node_function in node_definitions_v3:
    graph_builder_v3.add_node(node_name, node_function)

# 실행 흐름 정의 (V2와 동일한 구조)
graph_builder_v3.add_edge(START, "fetch_market")  # 시작 → 시장 데이터
graph_builder_v3.add_edge("fetch_market", "llm_decider")  # 시장 데이터 → LLM 결정

# LLM 결정에 따른 조건부 분기
graph_builder_v3.add_conditional_edges(
    "llm_decider",  # 소스: LLM 의사결정 노드
    route_by_action,  # 라우팅: action 값으로 분기
    {  # 분기 맵
        "buy": "buy",
        "sell": "sell",
        "hold": "hold"
    }
)

# 모든 실행 노드에서 종료
for execution_node in ["buy", "sell", "hold"]:
    graph_builder_v3.add_edge(execution_node, END)

# 실행 가능한 그래프로 컴파일
trading_graph_v3 = graph_builder_v3.compile()

# --- 데모 실행 ---
print("🤖 Trading Bot Graph V3 - LLM-Powered Strategy")
display(Image(trading_graph_v3.get_graph().draw_mermaid_png()))

print("=" * 55)

# MSFT 주식에 대한 LLM 기반 거래 시뮬레이션
demo_state = {
    "ticker": "MSFT",
    "logs": [],
    "messages": []
}

print(f"입력 상태: {demo_state}")
print("\n실행 결과:")

demo_result = trading_graph_v3.invoke(demo_state)
print(json.dumps(demo_result, ensure_ascii=False, indent=2, default=to_jsonable))

In [ ]:
set_debug(True)
# --- Tool Calling 버전: Prompt → ChatOpenAI(bind_tools) → tool_calls.args → TradingDecision ---
from typing import Literal
from pydantic import BaseModel, Field


class TradingDecision(BaseModel):
    action: Literal["BUY", "SELL", "HOLD"]
    reason: str = Field(..., description="결정에 대한 간단한 근거")


def create_llm_trading_chain_toolcalling():
    """
    OpenAI Tool Calling을 사용해 LLM이 함수 호출 형태로 결정값을 반환하도록 구성.
    Prompt → (ChatOpenAI.bind_tools([decision_tool]))
    """
    # 1) 프롬프트 (도구 호출을 유도)
    decision_prompt = PromptTemplate(
        template=(
            "You are a professional trading decision assistant.\n\n"
            "Ticker: {ticker}\n"
            "Current Market Trend: {trend}\n\n"
            "Choose EXACTLY one action from ['BUY','SELL','HOLD'] and provide a concise rationale.\n"
            "Call the tool to return your decision.\n"
        ),
        input_variables=["ticker", "trend"],
    )

    # 2) OpenAI tool 스키마 (Pydantic과 동일한 구조)
    decision_tool = {
        "type": "function",
        "function": {
            "name": "make_decision",
            "description": "Return a trading decision for the given ticker and trend.",
            "parameters": {
                "type": "object",
                "properties": {
                    "action": {"type": "string", "enum": ["BUY", "SELL", "HOLD"]},
                    "reason": {"type": "string", "description": "short rationale"}
                },
                "required": ["action", "reason"]
            }
        }
    }

    # 3) 모델 바인딩 (도구 호출 활성화)
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0).bind_tools([decision_tool])

    # 체인: 프롬프트 → LLM(도구 호출)
    return decision_prompt | llm


def llm_powered_decision_toolcalling(state: TraderState) -> dict:
    """
    Tool Calling으로 LLM이 호출한 함수의 args(JSON)를 받아 TradingDecision으로 검증 후 상태 업데이트.
    """
    chain = create_llm_trading_chain_toolcalling()
    t = state.get("trend", "flat")

    try:
        msg = chain.invoke({"ticker": state["ticker"], "trend": t})  # AIMessage
        # Tool call 결과 파싱
        tool_calls = getattr(msg, "tool_calls", None) or []
        if not tool_calls:
            # 도구 호출이 없으면 예외로 간주 (프롬프트 강화 또는 fallback)
            raise ValueError("No tool call returned by the model.")

        args = tool_calls[0].get("args", {})
        decision = TradingDecision.model_validate(args)

        return {"action": decision.action, **log(f"[llm_decider] {decision.reason}")}
    except Exception as e:
        return {"action": "HOLD", **log(f"[llm_decider] {e} -> fallback=HOLD")}


# --- 그래프 구성 (Tool Calling 버전) ---
graph_builder_v3_tools = StateGraph(TraderState)
for node_name, node_fn in [
    ("fetch_market", fetch_market_data),  # 이전 셀의 데이터 수집 노드 재사용
    ("llm_decider", llm_powered_decision_toolcalling),
    ("buy", execute_buy), ("sell", execute_sell), ("hold", execute_hold),
]:
    graph_builder_v3_tools.add_node(node_name, node_fn)

graph_builder_v3_tools.add_edge(START, "fetch_market")
graph_builder_v3_tools.add_edge("fetch_market", "llm_decider")
graph_builder_v3_tools.add_conditional_edges(
    "llm_decider", route_by_action, {"buy": "buy", "sell": "sell", "hold": "hold"}
)
for n in ["buy", "sell", "hold"]:
    graph_builder_v3_tools.add_edge(n, END)

trading_graph_v3_tools = graph_builder_v3_tools.compile()

# --- 데모 ---
print("🤖 Trading Bot Graph V3 - Tool Calling")
display(Image(trading_graph_v3_tools.get_graph().draw_mermaid_png()))
print("=" * 55)

demo_state = {"ticker": "MSFT", "logs": [], "messages": []}
print(f"입력 상태: {demo_state}\n\n실행 결과:")
demo_result = trading_graph_v3_tools.invoke(demo_state)
print(json.dumps(demo_result, ensure_ascii=False, indent=2, default=to_jsonable))

In [ ]:
from langchain_core.messages import ToolMessage
import json
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# 디버그 모드 비활성화
set_debug(False)


class TradingDecision(BaseModel):
    """
    거래 결정을 검증하기 위한 Pydantic 모델

    LLM이 tool call을 통해 전달한 매개변수의 유효성을 검증합니다.
    """
    action: Literal["BUY", "SELL", "HOLD"]
    reason: str = Field(..., description="거래 결정에 대한 간단한 근거")


@tool
def execute_trading_decision(action: Literal["BUY", "SELL", "HOLD"], reason: str) -> str:
    """
    거래 결정을 내리고 즉시 실행하는 통합 도구

    기존에 별도 노드들(buy/sell/hold)로 분리되어 있던 실행 로직을
    도구 내부에서 직접 처리합니다. 이렇게 하면 조건부 분기가 불필요해집니다.

    Args:
        action: 거래 행동 (BUY/SELL/HOLD)
        reason: 결정 근거

    Returns:
        실행 결과를 포함한 JSON 데이터
    """
    # 여기서 실제 거래 실행 로직 구현 가능:
    # - 포트폴리오 제약사항 체크
    # - 리스크 관리 규칙 적용
    # - 거래량 계산 및 주문 실행
    # - 거래소 API 호출
    # - 실행 결과 검증 등

    # 액션별 실행 로직
    if action == "BUY":
        execution_result = f"✅ 매수 주문이 성공적으로 실행되었습니다"
        execution_details = {
            "order_type": "market_buy",
            "status": "filled",
            "execution_price": 145.67,  # 실제로는 시장 가격
            "quantity": 100
        }
    elif action == "SELL":
        execution_result = f"✅ 매도 주문이 성공적으로 실행되었습니다"
        execution_details = {
            "order_type": "market_sell",
            "status": "filled",
            "execution_price": 145.23,  # 실제로는 시장 가격
            "quantity": 100
        }
    else:  # HOLD
        execution_result = f"📋 포지션을 유지합니다 (거래 없음)"
        execution_details = {
            "order_type": "no_action",
            "status": "held",
            "current_position": "maintained"
        }

    # 실행 결과 데이터 구성
    result_data = {
        "action": action,
        "reason": reason,
        "execution_result": execution_result,
        "execution_details": execution_details,
        "timestamp": "2024-08-25T10:00:00Z",
        "status": "completed"
    }

    return json.dumps(result_data, ensure_ascii=False, indent=2)


def create_unified_trading_tools():
    """
    통합된 거래 도구가 바인딩된 LLM을 생성하는 팩토리 함수

    Returns:
        도구가 연결된 ChatOpenAI 인스턴스와 ToolNode
    """
    unified_tools = [execute_trading_decision]

    # LLM에 통합 도구 바인딩
    llm_with_tools = ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=0
    ).bind_tools(unified_tools)

    # ToolNode: 통합 도구 실행
    unified_tool_executor = ToolNode(unified_tools)

    return llm_with_tools, unified_tool_executor


def llm_decide_and_execute(state: TraderState) -> dict:
    """
    LLM이 거래 결정과 실행을 한 번에 처리하도록 유도하는 노드

    Args:
        state: 현재 트레이딩 상태 (ticker, trend 필요)

    Returns:
        LLM의 AIMessage (tool_calls 포함)가 추가된 상태
    """
    # 통합 실행 프롬프트
    unified_prompt = PromptTemplate(
        template=(
            "You are a professional trading system that makes decisions and executes trades.\n\n"
            "MARKET DATA:\n"
            "- Ticker Symbol: {ticker}\n"
            "- Current Market Trend: {trend}\n\n"
            "INSTRUCTIONS:\n"
            "1. Analyze the market data\n"
            "2. Make a trading decision: BUY, SELL, or HOLD\n"
            "3. Provide clear reasoning for your decision\n"
            "4. Use the `execute_trading_decision` tool to both decide and execute the trade\n\n"
            "The tool will handle both the decision making and trade execution in one step.\n"
            "Provide a concise but thorough rationale for your choice."
        ),
        input_variables=["ticker", "trend"],
    )

    # LLM과 도구 생성
    llm_with_tools, _ = create_unified_trading_tools()

    # 프롬프트 실행
    current_trend = state.get("trend", "flat")
    prompt_text = unified_prompt.format(
        ticker=state["ticker"],
        trend=current_trend
    )

    # LLM이 통합 도구 호출
    ai_message = llm_with_tools.invoke(prompt_text)

    return {"messages": [ai_message]}


def process_execution_result(state: TraderState) -> dict:
    """
    통합 도구의 실행 결과를 처리하는 노드

    Args:
        state: ToolMessage가 포함된 상태

    Returns:
        최종 로그와 메시지가 업데이트된 상태
    """
    # 최근 ToolMessage에서 실행 결과 추출
    for message in reversed(state.get("messages", [])):
        if isinstance(message, ToolMessage) and message.name == "execute_trading_decision":
            try:
                # 실행 결과 파싱
                if isinstance(message.content, str):
                    execution_result = json.loads(message.content)
                else:
                    execution_result = message.content

                # 로그 메시지 생성
                action = execution_result.get("action", "UNKNOWN")
                reason = execution_result.get("reason", "No reason provided")
                result = execution_result.get("execution_result", "Execution completed")

                log_message = f"[unified_executor] {action}: {reason} → {result}"

                return {
                    **log(log_message),
                    **say(f"거래 완료: {result}")
                }

            except Exception as e:
                error_msg = f"실행 결과 처리 실패: {e}"
                return {
                    **log(f"[unified_executor] {error_msg}"),
                    **say("거래 실행 중 오류가 발생했습니다.")
                }

    # 실행 결과를 찾지 못한 경우
    return {
        **log("[unified_executor] 실행 결과를 찾을 수 없습니다"),
        **say("거래 실행 상태를 확인할 수 없습니다.")
    }


# --- 간소화된 그래프 구성 ---
# 통합 버전: 시장 데이터 → LLM 결정&실행 → 도구 실행 → 결과 처리 → 종료
graph_builder_unified = StateGraph(TraderState)

# 통합 도구 노드 생성
_, unified_trading_tool = create_unified_trading_tools()

# 노드 정의 (대폭 간소화!)
unified_node_definitions = [
    ("fetch_market", fetch_market_data),  # 시장 데이터 수집
    ("decide_and_execute", llm_decide_and_execute),  # LLM 결정 + 실행 호출
    ("execute_trade", unified_trading_tool),  # 툴노드 도구 실행
    ("process_result", process_execution_result),  # 결과 처리
]

# 노드 추가
for node_name, node_function in unified_node_definitions:
    graph_builder_unified.add_node(node_name, node_function)

# 선형 실행 흐름 (조건부 분기 제거!)
graph_builder_unified.add_edge(START, "fetch_market")
graph_builder_unified.add_edge("fetch_market", "decide_and_execute")
graph_builder_unified.add_edge("decide_and_execute", "execute_trade")
graph_builder_unified.add_edge("execute_trade", "process_result")
graph_builder_unified.add_edge("process_result", END)  # 바로 종료!

# 실행 가능한 그래프로 컴파일
trading_graph_unified = graph_builder_unified.compile()

# --- 데모 실행 ---
print("🤖 Trading Bot Graph - Unified ToolNode (No Branching)")
print("=" * 65)

# 그래프 시각화
print("간소화된 그래프 구조:")
display(Image(trading_graph_unified.get_graph().draw_mermaid_png()))

print("\n" + "=" * 65)

# 통합 실행 데모
demo_state = {
    "ticker": "AAPL",
    "logs": [],
    "messages": []
}

print(f"입력 상태: {demo_state}")
print("\n실행 결과:")

demo_result = trading_graph_unified.invoke(demo_state)
print(json.dumps(demo_result, ensure_ascii=False, indent=2, default=to_jsonable))

print("\n🎯 장점:")
print("- 조건부 분기 제거로 그래프 구조 단순화")
print("- buy/sell/hold 노드들을 하나의 통합 도구로 대체")
print("- 도구 내에서 모든 비즈니스 로직 처리 가능")
print("- 더 나은 오류 처리 및 로깅")

In [ ]:

from typing import Literal
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from random import choice
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage



class TradingDecision(BaseModel):
    """거래 결정을 검증하기 위한 Pydantic 모델"""
    action: Literal["BUY", "SELL", "HOLD"]
    reason: str = Field(..., description="거래 결정에 대한 간단한 근거")


@tool
def execute_trading_decision(action: Literal["BUY", "SELL", "HOLD"], reason: str) -> str:
    """
    거래 결정을 내리고 즉시 실행하는 도구

    실제 시스템에서는 여기서 포트폴리오 검증, 리스크 관리,
    거래소 API 호출 등의 로직을 구현할 수 있습니다.

    Args:
        action: 거래 행동 (BUY/SELL/HOLD)
        reason: 거래 근거

    Returns:
        거래 실행 결과 JSON 문자열
    """
    # 실제 거래 실행 로직 시뮬레이션
    if action == "BUY":
        execution_result = "매수 주문 실행 완료"
        execution_price = round(uniform(140, 160), 2)
    elif action == "SELL":
        execution_result = "매도 주문 실행 완료"
        execution_price = round(uniform(140, 160), 2)
    else:  # HOLD
        execution_result = "포지션 유지 (거래 없음)"
        execution_price = None

    # 실행 결과 구성
    result_data = {
        "action": action,
        "reason": reason,
        "execution_result": execution_result,
        "execution_price": execution_price,
        "status": "completed",
        "timestamp": "2024-08-25T10:00:00Z"
    }

    return json.dumps(result_data, ensure_ascii=False)


@tool()
def analyze_market_trend(ticker: str) -> str:
    """
    주식 심볼에 대한 시장 트렌드를 분석하는 도구

    실제 시스템에서는 API나 데이터 피드를 통해 실시간 시장 데이터를
    분석하여 트렌드를 판단합니다.

    Args:
        ticker: 주식 심볼 (예: "AAPL", "GOOGL")

    Returns:
        시장 분석 결과 JSON 문자열
    """
    # 랜덤 시장 데이터 및 트렌드 분석 (데모용)
    current_price = round(uniform(100, 200), 2)
    previous_price = round(current_price * uniform(0.95, 1.05), 2)
    price_change = current_price - previous_price
    price_change_pct = (price_change / previous_price) * 100

    # 트렌드 결정 로직
    if price_change_pct > 2:
        trend = "up"
        trend_strength = "strong"
    elif price_change_pct > 0.5:
        trend = "up"
        trend_strength = "moderate"
    elif price_change_pct < -2:
        trend = "down"
        trend_strength = "strong"
    elif price_change_pct < -0.5:
        trend = "down"
        trend_strength = "moderate"
    else:
        trend = "flat"
        trend_strength = "neutral"

    market_analysis = {
        "ticker": ticker,
        "current_price": current_price,
        "previous_price": previous_price,
        "price_change": round(price_change, 2),
        "price_change_pct": round(price_change_pct, 2),
        "trend": trend,
        "trend_strength": trend_strength,
        "volume": int(uniform(1000000, 5000000)),
        "volatility": round(uniform(0.1, 0.3), 3),
        "recommendation_confidence": round(uniform(0.7, 0.95), 2)
    }

    return json.dumps(market_analysis, ensure_ascii=False)


def create_trading_react_agent():
    """
    거래용 ReAct 에이전트를 생성하는 팩토리 함수

    Returns:
        설정된 create_react_agent 인스턴스
    """
    # LLM 모델 설정
    trading_model = ChatOpenAI(
        model="gpt-4.1-mini",  # 요청하신 모델로 설정
        temperature=0
    )

    # 사용 가능한 도구들
    trading_tools = [
        analyze_market_trend,  # 핵심: 시장 트렌드 분석
        execute_trading_decision,  # 핵심: 거래 결정 + 실행
    ]

    # create_react_agent로 간단하게 에이전트 생성
    # ReAct 패턴: Reasoning(추론) + Acting(행동)을 반복
    agent = create_react_agent(trading_model, tools=trading_tools)

    return agent


def run_trading_analysis(ticker: str):
    """
    ticker만으로 전체 거래 분석을 실행하는 함수

    Args:
        ticker: 주식 심볼 (예: "AAPL", "GOOGL")

    Returns:
        에이전트 실행 결과
    """
    # ReAct 에이전트 생성
    trading_agent = create_trading_react_agent()

    # ticker 기반 분석 프롬프트
    analysis_prompt = (
        f"You are a professional trading agent. Analyze and trade the stock: {ticker}\n\n"
        f"WORKFLOW:\n"
        f"1. First, use `analyze_market_trend('{ticker}')` to get current market trend\n"
        f"2. Based on the trend analysis, make a trading decision\n"
        f"3. Execute your decision using `execute_trading_decision(action, reason)`\n\n"
        f"Important: Always analyze the market trend first before making any trading decisions.\n"
        f"Provide clear reasoning based on the market data you retrieve."
    )

    # 에이전트 실행
    result = trading_agent.invoke({
        "messages": [HumanMessage(content=analysis_prompt)]
    })

    return result


def run_simple_trading_analysis(ticker: str):
    """
    간단한 버전: ticker 기반 빠른 분석 및 거래

    Args:
        ticker: 주식 심볼

    Returns:
        에이전트 실행 결과
    """
    trading_agent = create_trading_react_agent()

    simple_prompt = (
        f"Analyze and trade {ticker} stock:\n"
        f"1. Check market trend using analyze_market_trend\n"
        f"2. Make trading decision and execute immediately\n"
        f"Be concise but thorough in your analysis."
    )

    result = trading_agent.invoke({
        "messages": [HumanMessage(content=simple_prompt)]
    })

    return result


# --- 데모 실행 ---
print("🤖 Trading Bot - create_react_agent Version (Ticker-Only Input)")
print("=" * 60)

# create_react_agent도 결국 LangGraph이므로 구조 확인 가능
trading_agent = create_trading_react_agent()
print("create_react_agent 내부 노드 구조:")
display(Image(trading_agent.get_graph().draw_mermaid_png()))

print("\n" + "=" * 60)

# ticker만으로 전체 분석 실행
demo_ticker = "AAPL"

print(f"입력: ticker='{demo_ticker}' (trend 자동 분석)")
print("\n실행 결과:")

result = run_simple_trading_analysis(demo_ticker)

# 간결한 출력 - 핵심 메시지들만
print("ReAct 실행 과정:")
for i, msg in enumerate(result["messages"]):
    msg_type = type(msg).__name__
    if hasattr(msg, 'content') and msg.content:
        content_preview = msg.content[:80].replace('\n', ' ').strip()
        print(f"  {i + 1}. {msg_type}: {content_preview}...")

In [ ]:
"""
Trading Bot with Human-in-the-Loop (HITL) Approval System

This bot analyzes market trends and requires human approval for BUY/SELL decisions.
HOLD decisions are automatically executed without approval.

Key Features:
- Market trend analysis with simulated data
- Human approval required for BUY/SELL orders only
- Secure token-based approval system with TTL
- State management with checkpointing
"""

import time
import uuid
from typing import Literal, Dict, Any
from pydantic import BaseModel, Field
from random import uniform

from IPython.display import display, Image
from langchain_core.tools import tool
from langchain_core.messages import AIMessage
from langchain_openai import ChatOpenAI
from langgraph.types import Command, interrupt
from langchain_core.runnables import RunnableLambda

# LangGraph checkpointer compatibility
try:
    from langgraph.checkpoint.memory import MemorySaver
except ImportError:
    from langgraph.checkpoint.memory import InMemorySaver as MemorySaver

from langgraph.prebuilt import create_react_agent


# ============================================================================
# APPROVAL TOKEN MANAGEMENT (Server-side storage, inaccessible to LLM)
# ============================================================================

# Global token storage: {thread_id: expiry_timestamp}
# 스레드별로 최근 승인 토큰을 관리 (LLM에게 노출되지 않음)
thread_approval_tokens: Dict[str, float] = {}
TOKEN_EXPIRY_SECONDS = 60.0


def issue_thread_approval(thread_id: str) -> None:
    """Issue approval for a specific thread (server-side only)."""
    thread_approval_tokens[thread_id] = time.time() + TOKEN_EXPIRY_SECONDS


def validate_and_consume_thread_approval(thread_id: str) -> bool:
    """
    Validate and consume thread approval (one-time use).

    Args:
        thread_id: Thread identifier

    Returns:
        bool: True if thread has valid approval, False otherwise
    """
    current_time = time.time()
    expiry_time = thread_approval_tokens.get(thread_id)

    if expiry_time is None or expiry_time < current_time:
        # Remove expired/invalid approval
        thread_approval_tokens.pop(thread_id, None)
        return False

    # Consume approval (one-time use)
    thread_approval_tokens.pop(thread_id, None)
    return True


# ============================================================================
# DATA MODELS
# ============================================================================

class TradingDecision(BaseModel):
    """Model for validating trading decisions."""
    action: Literal["BUY", "SELL", "HOLD"]
    reason: str = Field(..., description="Brief reasoning for the trading decision")


# ============================================================================
# TRADING TOOLS
# ============================================================================

@tool("analyze_market_trend")
def analyze_market_trend(ticker: str) -> str:
    """
    Analyze market trend for a given stock ticker.

    Args:
        ticker: Stock symbol to analyze

    Returns:
        JSON string containing market analysis data
    """
    # Simulate market data
    current_price = round(uniform(100, 200), 2)
    previous_price = round(uniform(95, 105), 2)
    price_change = round(current_price - previous_price, 2)
    price_change_percent = round((price_change / previous_price) * 100, 2)

    # Determine trend based on price change
    if price_change_percent > 2:
        trend, strength = "upward", "strong"
    elif price_change_percent > 0.5:
        trend, strength = "upward", "moderate"
    elif price_change_percent < -2:
        trend, strength = "downward", "strong"
    elif price_change_percent < -0.5:
        trend, strength = "downward", "moderate"
    else:
        trend, strength = "sideways", "neutral"

    analysis_result = {
        "ticker": ticker,
        "current_price": current_price,
        "previous_price": previous_price,
        "price_change": price_change,
        "price_change_percent": price_change_percent,
        "trend_direction": trend,
        "trend_strength": strength,
    }

    return json.dumps(analysis_result, ensure_ascii=False)


@tool("request_user_approval")
def request_user_approval(
    ticker: str,
    action: Literal["BUY", "SELL", "HOLD"],
    reason: str
) -> str:
    """
    Request user approval for trading decisions.

    Args:
        ticker: Stock symbol
        action: Trading action (BUY/SELL/HOLD)
        reason: Reasoning for the decision

    Returns:
        JSON string with approval status (no token exposed to LLM)
    """
    if action in ("BUY", "SELL"):
        approval_request = {
            "ticker": ticker,
            "action": action,
            "reason": reason,
            "message": f"[승인 요청] {ticker} {action}\n근거: {reason}\n승인하시겠습니까? (yes/no)"
        }

        # Interrupt workflow for human approval
        user_response = interrupt(approval_request)

        # Simple approval check (detailed parsing handled in workflow)
        is_approved = bool(user_response.get("approved", False)) if isinstance(user_response, dict) else bool(user_response)

        if is_approved:
            # Issue server-side approval (no token sent to LLM)
            return json.dumps({
                "approved": True,
                "message": "승인 완료"
            }, ensure_ascii=False)

        return json.dumps({"approved": False}, ensure_ascii=False)

    # HOLD actions don't require approval
    return json.dumps({
        "approved": True,
        "auto_approved": True,
        "note": "HOLD 액션은 승인이 필요하지 않습니다"
    }, ensure_ascii=False)


@tool("execute_trade")
def execute_trade(
    ticker: str,
    action: Literal["BUY", "SELL", "HOLD"],
    reason: str,
) -> str:
    """
    Execute the trading decision.

    Args:
        ticker: Stock symbol
        action: Trading action
        reason: Decision reasoning

    Returns:
        JSON string with execution results

    Raises:
        RuntimeError: If approval is missing/invalid for BUY/SELL
    """
    # Validate approval for BUY/SELL actions (server-side check)
    if action in ("BUY", "SELL"):
        # Need to get thread_id from current execution context
        # This will be handled by the pre-model hook
        pass  # Validation happens in pre-model hook

    # Simulate trade execution
    if action == "BUY":
        execution_result = "매수 주문 체결 완료"
        execution_price = round(uniform(140, 160), 2)
    elif action == "SELL":
        execution_result = "매도 주문 체결 완료"
        execution_price = round(uniform(140, 160), 2)
    else:  # HOLD
        execution_result = "포지션 유지 (거래 없음)"
        execution_price = None

    trade_result = {
        "ticker": ticker,
        "action": action,
        "reason": reason,
        "execution_result": execution_result,
        "execution_price": execution_price,
        "status": "completed",
        "timestamp": time.time()
    }

    return json.dumps(trade_result, ensure_ascii=False)


# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def extract_last_ai_message(messages) -> str:
    """Extract the last AI message content from message history."""
    for message in reversed(messages or []):
        if isinstance(message, AIMessage):
            content = message.content
            return content if isinstance(content, str) else str(content)
    return ""


# ============================================================================
# PRE-MODEL HOOK: Approval Management
# ============================================================================

# Global variable to track current thread context
current_thread_id = None

def manage_approval_workflow(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Pre-model hook to manage approval workflow securely.

    This hook:
    1. Tracks thread context for approval validation
    2. Issues server-side approval when user approves
    3. Validates approval before trade execution
    4. Never exposes tokens to LLM
    """
    global current_thread_id

    try:
        messages = state.get("messages", [])

        # Find the most recent tool calls and responses
        for i, message in enumerate(reversed(messages)):
            tool_name = getattr(message, "name", None)

            # Handle approval issuance
            if tool_name == "request_user_approval":
                try:
                    response_data = json.loads(message.content)
                    if response_data.get("approved") and current_thread_id:
                        # Issue server-side approval for this thread
                        issue_thread_approval(current_thread_id)
                except (json.JSONDecodeError, AttributeError):
                    continue
                break

            # Handle trade execution validation
            elif tool_name == "execute_trade":
                # Look for the corresponding tool call (ToolMessage comes after AIMessage)
                tool_call_msg = messages[-(i+2)] if i+1 < len(messages) else None
                if hasattr(tool_call_msg, "tool_calls"):
                    for tool_call in tool_call_msg.tool_calls:
                        if tool_call["name"] == "execute_trade":
                            args = tool_call["args"]
                            action = args.get("action")

                            # Validate approval for BUY/SELL before execution
                            if action in ("BUY", "SELL") and current_thread_id:
                                if not validate_and_consume_thread_approval(current_thread_id):
                                    # Inject error into state to prevent execution
                                    raise RuntimeError("유효한 승인이 없습니다 (BUY/SELL 액션)")
                break

    except Exception as e:
        # Handle validation errors by preventing tool execution
        if "승인이 없습니다" in str(e):
            raise e
        # Other errors are silently handled
        pass

    return {}  # Don't modify agent state


def set_thread_context(thread_id: str):
    """Set the current thread context for approval tracking."""
    global current_thread_id
    current_thread_id = thread_id


# Create pre-model hook runnable
approval_management_hook = RunnableLambda(manage_approval_workflow)


# ============================================================================
# AGENT CONFIGURATION
# ============================================================================

SYSTEM_PROMPT = """You are a professional trading agent with human oversight.

Your workflow:
1. Analyze market trend using analyze_market_trend(ticker)
2. Make a trading decision (BUY/SELL/HOLD) with clear reasoning
3. For BUY/SELL: Call request_user_approval(ticker, action, reason)
   - Returns JSON with approval status
4. If approved: Call execute_trade(ticker, action, reason)
   - Server automatically validates approval (no token needed)
   - If not approved, respond with "거래가 거부되었습니다"
5. For HOLD: Skip approval and call execute_trade(ticker, action, reason)

Important:
- Always use valid JSON arguments for tools
- Minimize tool calls for efficiency
- Include reasoning for all decisions
- No need to handle approval tokens - server manages security
- Answer in Korean
"""

# Initialize components
memory_saver = MemorySaver()
chat_model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Create agent with tools and configuration
trading_agent = create_react_agent(
    model=chat_model,
    tools=[analyze_market_trend, request_user_approval, execute_trade],
    prompt=SYSTEM_PROMPT,
    checkpointer=memory_saver,
    pre_model_hook=approval_management_hook,
)

# Display agent graph if possible
try:
    agent_graph = trading_agent.get_graph(xray=True) if hasattr(trading_agent, "get_graph") else None
    if agent_graph is not None:
        if hasattr(agent_graph, "draw_mermaid_png"):
            display(Image(agent_graph.draw_mermaid_png()))
        elif hasattr(agent_graph, "draw_png"):
            display(Image(agent_graph.draw_png()))
except Exception as e:
    print(f"[그래프 표시 생략] {e}")


# ============================================================================
# WORKFLOW EXECUTION FUNCTIONS
# ============================================================================

def initiate_trading_analysis(thread_id: str, ticker: str) -> Dict[str, Any]:
    """
    Start trading analysis and run until human approval is needed.

    Args:
        thread_id: Unique thread identifier
        ticker: Stock symbol to analyze

    Returns:
        Dict containing interrupt payload and current state
    """
    # Set thread context for approval tracking
    set_thread_context(thread_id)

    config = {"configurable": {"thread_id": thread_id}}
    inputs = {"messages": [HumanMessage(content=f"거래 분석 대상: {ticker}")]}

    interrupt_payload = None

    # Stream until interrupt occurs
    for event in trading_agent.stream(inputs, config, stream_mode="updates"):
        if "__interrupt__" in event:
            interrupt_info = event["__interrupt__"][0]
            interrupt_payload = getattr(interrupt_info, "value", None)
            break

    current_state = trading_agent.get_state(config).values

    return {
        "interrupt_payload": interrupt_payload,
        "state": current_state
    }


def complete_with_approval(thread_id: str, approval_response) -> Dict[str, Any]:
    """
    Resume trading workflow with user approval decision.

    Args:
        thread_id: Thread identifier
        approval_response: User approval (bool, str, or dict)

    Returns:
        Dict containing final state and message
    """
    # Set thread context for approval tracking
    set_thread_context(thread_id)

    config = {"configurable": {"thread_id": thread_id}}

    # Normalize approval response
    if not isinstance(approval_response, dict):
        resume_data = {"approved": approval_response}
    else:
        resume_data = approval_response

    # Resume workflow with approval decision
    for _ in trading_agent.stream(Command(resume=resume_data), config, stream_mode="values"):
        pass  # Consume stream to completion

    final_state = trading_agent.get_state(config).values
    final_message = extract_last_ai_message(final_state.get("messages", []))

    return {
        "state": final_state,
        "final_message": final_message or "처리 완료"
    }


def run_interactive_trading(ticker: str) -> Dict[str, Any]:
    """
    Run complete trading workflow with interactive approval.

    Args:
        ticker: Stock symbol to trade

    Returns:
        Dict containing execution results
    """
    thread_id = f"trading-{uuid.uuid4().hex[:8]}"

    print(f"🔍 {ticker} 거래 분석을 시작합니다...")

    # Step 1: Start analysis
    analysis_result = initiate_trading_analysis(thread_id, ticker)

    if analysis_result["interrupt_payload"] is None:
        # Completed without interruption (likely HOLD)
        final_message = extract_last_ai_message(analysis_result["state"].get("messages", []))
        return {
            "approved": True,
            "status": "completed_without_approval",
            "final_message": final_message or "거래 분석 완료"
        }

    # Step 2: Request user approval
    approval_request = analysis_result["interrupt_payload"]
    print(f"\n📋 {approval_request.get('message', '승인이 필요합니다.')}")

    user_input = input("👤 승인 여부를 입력하세요 (yes/no): ").strip().lower()
    is_approved = user_input in ("yes", "y", "승인", "ok")

    # Step 3: Complete workflow
    completion_result = complete_with_approval(thread_id, is_approved)

    return {
        "approved": is_approved,
        "status": "completed",
        "final_message": completion_result["final_message"]
    }


# ============================================================================
# DEMO EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("🤖 Trading Bot - Human-in-the-Loop (HITL) 시스템")
    print("=" * 60)

    # Demo 1
    print("\n📊 데모 #1 (NVDA):")
    demo_result_1 = run_interactive_trading("NVDA")
    print(f"✅ 결과: {demo_result_1['final_message']}")

    print("\n" + "=" * 60)

    # Demo 2
    print("\n📊 데모 #2 (AAPL):")
    demo_result_2 = run_interactive_trading("AAPL")
    print(f"✅ 결과: {demo_result_2['final_message']}")

In [ ]:
"""
30분 완성 간단 에이전트 아이디어 10개

1. 날씨 기반 옷차림 추천 에이전트
도구: 날씨 조회, 옷차림 추천
입력: 지역, 외출 시간
출력: 날씨 정보 + 적합한 옷차림 조언

2. 간단 계산기 에이전트
도구: 기본 계산, 단위 변환
입력: 수식 또는 계산 요청
출력: 계산 결과 + 설명

3. 영어 단어 학습 도우미
도구: 단어 정의 검색, 예문 생성
입력: 영어 단어
출력: 뜻, 발음, 예문

4. 간단 할일 관리자
도구: 할일 추가, 완료 체크, 목록 조회
입력: 할일 내용
출력: 업데이트된 할일 목록

5. 음식 칼로리 조회 에이전트
도구: 칼로리 검색, 영양 정보 조회
입력: 음식명
출력: 칼로리 정보 + 영양 성분

6. 간단 번역 에이전트
도구: 텍스트 번역, 언어 감지
입력: 번역할 텍스트
출력: 번역 결과 + 원문 언어

7. 책 추천 에이전트
도구: 장르별 검색, 평점 조회
입력: 선호 장르, 기분
출력: 추천 도서 목록

8. 간단 퀴즈 출제자
도구: 문제 생성, 정답 확인
입력: 퀴즈 주제
출력: 문제 + 정답 확인

9. 색상 조합 추천 에이전트
도구: 색상 분석, 조합 생성
입력: 기본 색상
출력: 어울리는 색상 조합

10. 간단 메모 정리 에이전트
도구: 메모 분류, 키워드 추출
입력: 긴 텍스트 메모
출력: 정리된 메모 + 주요 키워드
"""

In [ ]:
# Simple Agent Template - create_react_agent 기반
# 30분 완성용 에이전트 개발 템플릿

import os
import json
from typing import Literal
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from getpass import getpass
from langchain_core.globals import set_debug


# =============================================================================
# 설정: OpenAI API 키
# =============================================================================
if not os.getenv("OPENAI_API_KEY"):
    print("🔑 OPENAI_API_KEY 입력 필요")
    key = getpass("API Key: ").strip()
    if key:
        os.environ["OPENAI_API_KEY"] = key
        print("✅ API Key 설정 완료")
else:
    print("✅ API Key 확인됨")

# =============================================================================
# 1단계: 데이터 모델 정의 (필요시)
# =============================================================================
class MyDataModel(BaseModel):
    """에이전트에서 사용할 데이터 구조"""
    result: str = Field(..., description="처리 결과")
    status: Literal["success", "error"] = "success"

# =============================================================================
# 2단계: 도구(Tools) 정의
# =============================================================================

@tool("tool_one")
def my_first_tool(input_text: str) -> str:
    """
    첫 번째 도구 설명

    Args:
        input_text: 입력 텍스트

    Returns:
        처리 결과 문자열
    """
    # 여기에 실제 로직 구현
    result = f"처리 완료: {input_text}"
    return result

@tool("tool_two")
def my_second_tool(number: int) -> str:
    """
    두 번째 도구 설명

    Args:
        number: 입력 숫자

    Returns:
        계산 결과
    """
    # 여기에 실제 로직 구현
    result = f"계산 결과: {number * 2}"
    return result

# 추가 도구들을 여기에 구현...

# =============================================================================
# 3단계: 시스템 프롬프트 정의
# =============================================================================
SYSTEM_PROMPT = """
당신은 [에이전트의 역할]을 담당하는 도우미입니다.

주요 기능:
1. [기능 1 설명]
2. [기능 2 설명]
3. [기능 3 설명]

사용 가능한 도구:
- tool_one: [도구 1 설명]
- tool_two: [도구 2 설명]

작업 순서:
1. 사용자 요청 분석
2. 적절한 도구 선택 및 실행
3. 결과를 사용자 친화적으로 정리하여 응답

항상 한국어로 정중하게 답변해주세요.
"""

# =============================================================================
# 4단계: 에이전트 생성
# =============================================================================
def create_my_agent():
    """에이전트 생성 함수"""

    # LLM 모델 설정
    llm = ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=0.1
    )

    # 도구 목록
    tools = [
        my_first_tool,
        my_second_tool,
        # 추가 도구들...
    ]

    # create_react_agent로 에이전트 생성
    agent = create_react_agent(
        model=llm,
        tools=tools,
        prompt=SYSTEM_PROMPT
    )

    set_debug(False) # 자세한 디버깅을 원하면 여기를 True

    return agent

# =============================================================================
# 5단계: 실행 함수들
# =============================================================================

def run_agent(user_input: str) -> str:
    """
    에이전트 실행 함수

    Args:
        user_input: 사용자 입력

    Returns:
        에이전트 응답
    """
    agent = create_my_agent()

    # 에이전트 실행
    result = agent.invoke({
        "messages": [HumanMessage(content=user_input)]
    })

    # 마지막 메시지에서 응답 추출
    if result.get("messages"):
        last_message = result["messages"][-1]
        return last_message.content

    return "응답을 생성할 수 없습니다."

def interactive_mode():
    """대화형 모드 실행"""
    print("=== 간단 에이전트 대화형 모드 ===")
    print("종료하려면 'quit' 입력")

    previous_response = ""

    while True:
        # 이전 응답을 input 창의 프롬프트에 포함
        if previous_response:
            prompt = f"\n[이전 응답]\n {previous_response}\n --- \n사용자: "
        else:
            prompt = "\n사용자: "

        user_input = input(prompt).strip()

        if user_input.lower() in ['quit', 'exit', '종료']:
            print("에이전트를 종료합니다.")
            break

        if not user_input:
            continue

        try:
            response = run_agent(user_input)
            print(f"에이전트: {response}")
            # 다음 input에서 사용할 수 있도록 저장
            previous_response = response
        except Exception as e:
            error_msg = f"오류가 발생했습니다: {e}"
            print(error_msg)
            previous_response = error_msg


# =============================================================================
# 개발 가이드
# =============================================================================
"""
이 템플릿 사용법:

1. 에이전트 아이디어 선택
   - 위의 10개 아이디어 중 하나 선택하거나 새로운 아이디어 구상

2. 도구(Tools) 구현
   - @tool 데코레이터를 사용해 필요한 기능들을 함수로 구현
   - 각 도구는 명확한 입력/출력 타입과 설명을 가져야 함

3. 시스템 프롬프트 수정
   - SYSTEM_PROMPT를 에이전트의 역할에 맞게 수정
   - 사용 가능한 도구들과 작업 플로우 명시

4. 테스트 케이스 작성
   - run_demo() 함수의 test_cases에 실제 사용 시나리오 추가

5. 실행 및 개선
   - python 파일명.py로 실행
   - 결과를 보고 도구나 프롬프트 개선

팁:
- 도구는 단순하고 명확한 기능만 구현
- 에러 처리를 잊지 말 것
- 사용자 친화적인 메시지 출력
- 필요시 외부 API나 라이브러리 활용
"""

# =============================================================================
# 실행 부분
# =============================================================================

if __name__ == "__main__":
    # 대화형 모드
    interactive_mode()

